In [1]:
import flatbuffers
import numpy as np
import FlatBufTaskGraph.TaskGraph

In [21]:
buf = []
with open("/home/weiyangw/Documents/tg_d4_b128.fbuf", "rb") as f:
    buf = f.read()
tg = FlatBufTaskGraph.TaskGraph.TaskGraph.GetRootAs(buf, 0)

In [3]:
def get_all_routes(tg):
    result = {}
    for i in range(tg.RoutesLength()):
        r = tg.Routes(i)
        src, dst = r.Fromnode(), r.Tonode()
        for j in range(r.PathsLength()):
            p = r.Paths(j)
            path = [p.Hopnode(k) for k in range(p.HopnodeLength())]
            result[(src,dst)] = path
    return result
        

In [4]:
DEVICE_COMP_GPU = 0
DEVICE_COMP_CPU = 1
DEVICE_MEM_SYSTEM = 2
DEVICE_MEM_Z_COPY = 3
DEVICE_MEM_GPU_FB = 4
DEVICE_COMM_MEMBUS_COMM = 5
DEVICE_COMM_UPI_IN_COMM = 6
DEVICE_COMM_UPI_OUT_COMM = 7
DEVICE_COMM_NIC_IN_COMM = 8
DEVICE_COMM_NIC_OUT_COMM = 9
DEVICE_COMM_PCI_TO_HOST_COMM = 10
DEVICE_COMM_PCI_TO_DEV_COMM = 11
DEVICE_COMM_NVLINK_COMM = 12
DEVICE_COMM_NW_COMM = 13
DEVICE_COMM_NW_NOMINAL = 14
def get_dev(tg):
    result = {}
    for i in range(tg.DevicesLength()):
        d = tg.Devices(i)
        result[d.Deviceid()] = (d.Nodeid(), d.Type(), d.Deviceproperty(), d.Bandwidth())
    return result

def get_nwnominal(tg, devices):
    nnodes = tg.Nnode()
    nominal_comms = {}
    for devid, dev in devices.items():
        if dev[1] == DEVICE_COMM_NW_NOMINAL:
            src, dst = dev[2] // nnodes, dev[2] % nnodes
            nominal_comms[devid] = (src, dst)
    return nominal_comms

In [5]:
TASK_FORWARD = 0
TASK_BACKWARD = 1
TASK_COMM = 2
TASK_UPDATE = 3
TASK_BARRIER = 4
TASK_NOMINAL_COMM = 5
TASK_ALLREDUCE = 6
def get_tasks(tg):
    result = {}
    for i in range(tg.TasksLength()):
        t = tg.Tasks(i)
        nexttasks = [t.Nexttasks(j) for j in range(t.NexttasksLength())]
        result[t.Taskid()] = (t.Deviceid(), t.Runtime(), t.Xfersize(), t.Type(), nexttasks)
    return result

In [6]:
def get_ring(tg):
    rings = {}
    for i in range(tg.RingsLength()):
        r = tg.Rings(i)
        rsz = r.Ringsz()
        rings[rsz] = [list(r.Ringpaths(j).JumpsAsNumpy()) for j in range(r.RingpathsLength())]
    return rings

In [7]:
def get_logical_tm(tasks, rings, nom_devs, tg):
    nnodes = tg.Nnode()
    result = {} # (src, dst) -> traffic
    tot = 0
    totr = 0
    for tid, t in tasks.items():
        if t[3] == TASK_NOMINAL_COMM:
            ndev = nom_devs[t[0]]
            # print(ndev, t[2])
            if ndev in result:
                result[ndev] += t[2]
            else:
                result[ndev] = t[2]
            tot += t[2]
        elif t[3] == TASK_ALLREDUCE:
            ringsz = len(t[-1])
            if ringsz == 1: 
                continue
            nsplit = len(rings[ringsz])
            totr += 2 * (len(t[-1]) - 1) * t[2]
            xfersize = 2 * (len(t[-1]) - 1) * t[2] / nsplit / len(t[-1])
            curr_node = t[-1][0]
            for rdesc in rings[ringsz]:
                total_hop = sum(rdesc)
                for j in range(len(t[-1])):
                    ndev = (curr_node, (curr_node + total_hop) % nnodes)
                    if ndev in result:
                        result[ndev] += xfersize
                    else:
                        result[ndev] = xfersize
                    curr_node = (curr_node + total_hop) % nnodes
    print(totr, tot)
    return result
            

In [8]:
def get_physical_tm(tasks, rings, nom_devs, routes, tg):
    nnodes = tg.Nnode()
    result = {}
    for tid, t in tasks.items():
        if t[3] == TASK_NOMINAL_COMM:
            ndev = nom_devs[t[0]]
            path = routes[ndev]
            for j in range(len(path) - 1):
                npdev = (path[j], path[(j + 1)])
                # print(npdev)
                if npdev in result:
                    result[npdev] += t[2]
                    # print(t, npdev, result[npdev])
                else:
                    result[npdev] = t[2]
        elif t[3] == TASK_ALLREDUCE:
            ringsz = len(t[-1])
            if ringsz == 1: 
                continue
            nsplit = len(rings[ringsz])
            xfersize = 2 * (len(t[-1]) - 1) * t[2] / nsplit / len(t[-1])
            # print(t, nsplit, xfersize)
            curr_node = t[-1][0]
            for rdesc in rings[ringsz]:
                # print(rdesc)print(t[2])
                for j in range(len(t[-1])):
                    for l in rdesc:
                        npdev = (curr_node, (curr_node + l) % nnodes)
                        # print(l, npdev)
                        if npdev in result:
                            result[npdev] += xfersize
                        else:
                            result[npdev] = xfersize
                        curr_node = (curr_node + l) % nnodes
    return result

In [9]:
def get_hop_to_traffic(tasks, rings, nom_devs, routes, tg):
    nnodes = tg.Nnode()
    result = {}
    for tid, t in tasks.items():
        if t[3] == TASK_NOMINAL_COMM:
            ndev = nom_devs[t[0]]
            path = routes[ndev]
            pathlen = len(path) - 1
            if pathlen in result:
                result[pathlen] += t[2]
            else:
                result[pathlen] = t[2]
        elif t[3] == TASK_ALLREDUCE:
            ringsz = len(t[-1])
            if ringsz == 1: 
                continue
            nsplit = len(rings[ringsz])
            xfersize = 2 * (len(t[-1]) - 1) * t[2] / nsplit / len(t[-1])
            # print(xfersize)
            for rdesc in rings[ringsz]:
                hoplen = len(rdesc)
                for j in range(len(t[-1])):
                    if hoplen in result:
                        result[hoplen] += xfersize
                    else:
                        result[hoplen] = xfersize
    return result

In [22]:
tasks = get_tasks(tg)
rings = get_ring(tg)
devs = get_dev(tg)
ndevs = get_nwnominal(tg, devs)
routes = get_all_routes(tg)
r = get_hop_to_traffic(tasks, rings, ndevs, routes, tg)
a = sum([k * v for k, v in r.items() if k != 1]) / r[1]
print(a)

0.2755831801975861


In [23]:
all_route_cnt = {}
for v in routes.values():
    for i in range(len(v)-1):
        link = (v[i], v[i+1])
        if link in all_route_cnt:
            all_route_cnt[link] += 1
        else:
            all_route_cnt[link] = 1
distribution = [(v,k) for k, v in all_route_cnt.items()]
sorted(distribution)

[(90, (37, 24)),
 (93, (39, 52)),
 (95, (33, 46)),
 (96, (10, 9)),
 (98, (2, 3)),
 (101, (47, 34)),
 (101, (116, 103)),
 (102, (9, 8)),
 (103, (97, 84)),
 (106, (2, 117)),
 (106, (50, 37)),
 (109, (28, 41)),
 (109, (60, 47)),
 (111, (54, 41)),
 (111, (63, 50)),
 (111, (70, 57)),
 (112, (38, 25)),
 (113, (5, 4)),
 (113, (24, 11)),
 (114, (11, 10)),
 (114, (15, 2)),
 (115, (8, 7)),
 (115, (32, 19)),
 (115, (40, 27)),
 (116, (4, 119)),
 (117, (0, 115)),
 (117, (119, 106)),
 (118, (67, 54)),
 (118, (103, 90)),
 (119, (3, 2)),
 (121, (85, 72)),
 (122, (4, 17)),
 (122, (25, 12)),
 (122, (30, 29)),
 (122, (73, 60)),
 (122, (77, 64)),
 (123, (41, 28)),
 (124, (9, 124)),
 (124, (56, 43)),
 (125, (11, 12)),
 (125, (65, 52)),
 (125, (76, 63)),
 (125, (90, 77)),
 (125, (93, 92)),
 (126, (6, 5)),
 (127, (21, 8)),
 (127, (46, 59)),
 (128, (5, 120)),
 (128, (17, 4)),
 (128, (18, 17)),
 (128, (57, 70)),
 (129, (12, 11)),
 (129, (14, 1)),
 (129, (15, 28)),
 (129, (16, 3)),
 (129, (44, 31)),
 (129, (110

In [12]:
tasks = get_tasks(tg)
rings = get_ring(tg)
devs = get_dev(tg)
ndevs = get_nwnominal(tg, devs)
routes = get_all_routes(tg)
# ptm = get_physical_tm(tasks, rings, ndevs,routes , tg)
ltm = get_logical_tm(tasks, rings, ndevs , tg)
# print(ptm)
# print(max([v for v in get_physical_tm(tasks, rings, ndevs,routes , tg).values() if v != 0]), min([v for v in get_physical_tm(tasks, rings, ndevs,routes , tg).values() if v != 0]))
# print(max([v for v in get_logical_tm(tasks, rings, ndevs, tg).values() if v != 0]), min([v for v in get_logical_tm(tasks, rings, ndevs , tg).values() if v != 0]))

342032794616 142807662592


In [14]:
tasks = get_tasks(tg)
rings = get_ring(tg)
devs = get_dev(tg)
ndevs = get_nwnominal(tg, devs)
routes = get_all_routes(tg)
r = get_hop_to_traffic(tasks, rings, ndevs, routes, tg)

In [25]:
def cdf(m):
    n = 0
    res = []
    total = sum(m.values())
    for i in sorted(m):
        n += m[i]
        res.append(n/total)
    
    return res

def cpdf(m):
    n = 0
    res = []
    total = sum(m.values())
    for i in sorted(m):
        n += m[i]
        total+=
        res.append(n/total*i)
    
    return res

In [24]:
for i in [64, 128, 256, 512, 1024]:
    buf = []
    with open(f"/home/weiyangw/Documents/tg_d8_b{i}.fbuf", "rb") as f:
        buf = f.read()
    tg = FlatBufTaskGraph.TaskGraph.TaskGraph.GetRootAs(buf, 0)
    tasks = get_tasks(tg)
    rings = get_ring(tg)
    devs = get_dev(tg)
    ndevs = get_nwnominal(tg, devs)
    routes = get_all_routes(tg)
    r = get_hop_to_traffic(tasks, rings, ndevs, routes, tg)
    print(",".join([str(x) for x in cdf(r)]))

0.9774266761500128,0.9831174300617743,0.9929814035088275,0.9992412328117651,1.0
0.9559153884273898,0.9670291560507369,0.9862930199312052,0.998518164316887,1.0
0.9099622198411313,0.9368424072541779,0.9737434726787032,0.9971614565058058,1.0
0.83776034035679,0.8818313377915413,0.9519534010800773,0.994805773089738,1.0
0.7254541413178842,0.7984687446532922,0.9180587203535364,0.9911414832814633,1.0


In [26]:
for i in [64, 128, 256, 512, 1024]:
    buf = []
    with open(f"/home/weiyangw/Documents/tg_d8_b{i}.fbuf", "rb") as f:
        buf = f.read()
    tg = FlatBufTaskGraph.TaskGraph.TaskGraph.GetRootAs(buf, 0)
    tasks = get_tasks(tg)
    rings = get_ring(tg)
    devs = get_dev(tg)
    ndevs = get_nwnominal(tg, devs)
    routes = get_all_routes(tg)
    r = get_hop_to_traffic(tasks, rings, ndevs, routes, tg)
    print(",".join([str(x) for x in ccdf(r)]))

0.9774266761500128,1.9662348601235486,2.9789442105264823,3.9969649312470605,5.0
0.9559153884273898,1.9340583121014738,2.958879059793616,3.994072657267548,5.0
0.9099622198411313,1.8736848145083558,2.9212304180361097,3.988645826023223,5.0
0.83776034035679,1.7636626755830827,2.855860203240232,3.979223092358952,5.0
0.7254541413178842,1.5969374893065844,2.7541761610606095,3.9645659331258534,5.0
